# 导入包

In [ ]:
import time

# Editor: WangNing
# Date:2024/12/11
import torch
import torchvision.transforms as transforms
import torchvision
import matplotlib
from model import optical_unit,DNN1
from utils import gen_SLM_mask
from Experimental_data_processing import classification_data_process as cdp
import glob
import sys
from DMD6500 import pycrafter6500
from utils import utils, patterns, find_center
from UPO_SLM_80Rplus.SLM_UPOLabs import *
from DaHeng.GetImage import *
matplotlib.use("TkAgg")
import matplotlib.pyplot as plt
get_ipython().run_line_magic('matplotlib', 'inline')

# 定义系统参数

In [ ]:
def gen_coor(sps):
    c = []
    l = sps['layer_params']
    for i in range(len(l)):
        c.append(optical_unit.FreDomainCoordinate(l[i]['Ny_s'], l[i]['Ny_e'], l[i]['Nx_s'],l[i]['Nx_e'], l[i]['py'], l[i]['px'], device))
    return c
device = "cuda" if torch.cuda.is_available() else "cpu"
mps = { # model parameters
    "layers_num": 3,  # hidden layer数
    "residual_num": [],  # residual layer 编号列表
    'residual_ratio': 1,  # 残差比例
    "model_path": "test.pth",  # 模型存储路径
    "dropout": 0,  # dropout比例
    # "class_region_centers": [[0.2, 0.2], [0.5, 0.2], [0.8, 0.2]],
    "class_region_centers": [[0.3, 0.25], [0.5, 0.25], [0.7, 0.25],
                             [0.2, 0.5], [0.4, 0.5], [0.6, 0.5], [0.8, 0.5],
                             [0.3, 0.75], [0.5, 0.75], [0.7, 0.75]
                             ],         # 不同类区域的中心坐标(百分比,(x,y))
    "class_region_pixels": 3  # 分类区域单边大小
}
scale_factor = 1  # 整体放缩因子，一般为1
inp_params = {
        'Ny_s': int(-100*scale_factor),
        'Ny_e': int(99*scale_factor),
        'Nx_s': int(-100*scale_factor),
        'Nx_e': int(99*scale_factor),
        'px': 7.56e-6/scale_factor,
        'py': 7.56e-6/scale_factor
    }  # 输入面大小, [y,x]
slm_params = {
        'Ny_s': int(-100*scale_factor),
        'Ny_e': int(99*scale_factor),
        'Nx_s': int(-100*scale_factor),
        'Nx_e': int(99*scale_factor),
        'px': 8e-6/scale_factor,
        'py': 8e-6/scale_factor
    }
cam_params = {
        'Ny_s': int(-100*scale_factor),
        'Ny_e': int(99*scale_factor),
        'Nx_s': int(-100*scale_factor),
        'Nx_e': int(99*scale_factor),
        'px': 5.86e-6/scale_factor,
        'py': 5.86e-6/scale_factor
    }
sps = {  # system parameters
    "lambda": torch.tensor(532e-9, device=device),  # 波长，单位都是m
    "distance": [200e-3,]*4,  # 层间距
    "method": ['Bluestein',]*4,  # 传播方法 APM, SAPM, Bluestein
    "image_pixels": 100,
    "layer_params": [inp_params]+[slm_params]*mps["layers_num"]+[cam_params],
    "shiftx": [0,0,0,0],
    "shifty": [0,0,0,0],
    "discretion": False,  # 是否对slm相位离散化
    "discretion_bits": 10,  # 离散化位深
}
coor = gen_coor(sps)

# 部署准备

#### 取出Mask

In [ ]:
model = DNN1.Dmodel(sps, mps, device, coor)
optimizer = torch.optim.Adam(model.parameters(), lr=0)
diff_nn = DNN1.DNN(device, sps, mps, coor, model, optimizer=optimizer)
diff_nn.load_checkpoints(mps["model_path"])
masks = model.visual_mask()
slm_mask = gen_SLM_mask.gen_multi_SLM_mask(SLM_size=(1200, 1920), bits=8, masks=masks, shiftx_list=[-400, 0, 400], shifty_list=[0,0,0])
slm_mask_np = slm_mask.cpu().numpy()
image = Image.fromarray(slm_mask_np)
image.show()
SLM_path = f"./SLM_pattern_8bits/{mps['model_path'][:-4]}.bmp"
image.save(SLM_path)

#### 取出MNIST图像，并旋转45°以适应DMD

In [ ]:
N = 200  # 取出N张图片
theta = torch.tensor(12*torch.pi/180)   # 由于DMD出射光与DMD平面不垂直，因此应该对图像进行旋转后的拉伸，角度24°。
transform = transforms.Compose(
    [transforms.Resize(size=(sps["image_pixels"], int(sps["image_pixels"]/torch.cos(theta)))),
     transforms.RandomRotation((45, 45)),     # 旋转45°匹配DMD角度
     transforms.ToTensor(),
     transforms.Lambda(utils.binarize)   # 二值化匹配DMD
     ])
dataset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)
dataset = utils.filter_dataset(dataset, [0, 1, 2, ])
ind = torch.randperm(len(dataset))[:N]
mini_dataset = [dataset[i] for i in ind]
os.makedirs(name="./DMD_MNIST_images", exist_ok=True)       # 创建文件夹
for i, (img, label) in enumerate(mini_dataset):
    img = utils.convert_to_DMDsize(img.squeeze())
    image = Image.fromarray(img.numpy())
    image.save(f"./DMD_MNIST_images/{i}{label}.bmp")        # 文件名最后一位存储label

# 部署

In [ ]:
slm = SLM_UP()
slm.Open_window(screenNum=1)
resX, resY = slm.Get_size(1)    # 得到slm的分辨率
slm.Set_Offset(screenNum=1, offset_x=0, offset_y=0)#图片平移量设置-3,-12
slm.Disp_ReadImage(path=os.path.abspath(SLM_path), screenNum=1, bits=8)  #平移后图片显示

In [ ]:
dlp = pycrafter6500.dmd()
# 0:video mode  1:Pre-stored Pattern Mode   2:Video Pattern Mode    3:Pattern On-The-Fly Mode
dlp.changemode(mode=3)

In [ ]:
# 只显示一张图片
folder_path = "./DMD_MNIST_images"
image_files = glob.glob(os.path.join(folder_path, '*.bmp'))
imgs = []
labels = []
for image_file in image_files:
    try:
        i = Image.open(image_file)
        imgs.append(np.array(i, dtype=np.uint8))
        print(type(imgs[0][0,0]))
        labels.append(image_file[-5])
    except Exception as e:
        print(f"Failed to open image: {image_file}, Error: {e}")
print(labels)
'''
:param images:图像
:param exp: 曝光时间
:param ti: 触发输入
:param dt: 黑暗时间
:param to: 触发输出
:param rep: 重复，0为一直重复
:return:
'''
dlp.stopsequence()  # 如果正在运行，需要先停止。
exposure_time = [100000]*len(imgs)  # 单位us
dark_time=[105]*len(imgs)        # 单位us,如果要用触发模式，darktime不能为0。
trigger_in=[False]*len(imgs)
trigger_out=[1]*len(imgs)   # 好像没用，设置为0或者1都会有硬件触发输出。
rep = 200  # 是显示多少张图，而不是整个序列重复多少次。
dlp.defsequence(images=imgs,exp=exposure_time,ti=trigger_in, dt=dark_time,to=trigger_out,rep=rep)

### 相机

In [ ]:
device_manager = gx.DeviceManager()
dev_num, dev_info_list = device_manager.update_all_device_list()
if dev_num is 0:
    print("Number of enumerated devices is 0")
# open the first device
cam = device_manager.open_device_by_index(1)
remote_device_feature = cam.get_remote_device_feature_control()

In [ ]:
# get image convert obj
# global image_convert
image_convert = device_manager.create_image_format_convert()
trigger_mode_feature = remote_device_feature.get_enum_feature("TriggerMode")
if dev_info_list[0].get("device_class") == gx.GxDeviceClassList.USB2:
    # set trigger mode
    trigger_mode_feature.set("On")
else:
    # set trigger mode and trigger source
    trigger_mode_feature.set("On")
    trigger_source_feature = remote_device_feature.get_enum_feature("TriggerSource")
    trigger_source_feature.set("Line0")
remote_device_feature = cam.get_remote_device_feature_control()
# 设置曝光值
exposure_time_feature = remote_device_feature.get_float_feature("ExposureTime")
# 获取曝光值可设置范围和最大值
float_range = exposure_time_feature.get_range()
float_max = float_range["max"]
# 设置当前曝光值范围内任意值
exposure_time_feature.set(50000.0)   # us
# 获取当前曝光值
float_exposure_value = exposure_time_feature.get()
print(f"exposure_value: {float_exposure_value}")

In [ ]:
# start data acquisition
cam.stream_on()

In [ ]:
np_imgs = []
# 提取文件名，去掉路径部分
image_filenames = [os.path.basename(file) for file in image_files]
dlp.startsequence()
for i in range(len(image_filenames)):
    np_imgs.append(capture(cam, image_convert))

In [ ]:
results_path = f"./results/20241103/test/"
for i in range(len(np_imgs)):
    img = Image.fromarray(np_imgs[i])
    img.save(f"{results_path}{image_filenames[i]}")

# 数据处理

### 获取探测区域

In [ ]:
region_path = f"./results/20241103/region/"
trigger_mode_feature.set("Off")
exposure_time_feature.set(20.0)   # us
image_file = ""
imgs = [np.array(Image.open(image_file), dtype=np.uint8)]
dlp.stopsequence()  # 如果正在运行，需要先停止。
exposure_time = [100000]*len(imgs)  # 单位us
dark_time=[0]*len(imgs)        # 单位us,如果要用触发模式，darktime不能为0。
trigger_in=[False]*len(imgs)
trigger_out=[1]*len(imgs)   # 好像没用，设置为0或者1都会有硬件触发输出。
rep = 0  # 是显示多少张图，而不是整个序列重复多少次。
dlp.defsequence(images=imgs,exp=exposure_time,ti=trigger_in, dt=dark_time,to=trigger_out,rep=rep)
centers = []
for i in range(mps["class_region_centers"]):
    pattern = patterns.focal_patterns_on_target(f=200e-3, coor=coor, focal_ratios=mps["class_region_centers"], lamb=sps['lambda'], region_size=200)
    slm.Disp_ReadImage(path=pattern, screenNum=1, bits=8)
    time.sleep(0.5)
    img = capture(cam, image_convert)
    img = Image.fromarray(img)
    img.save(f"{region_path}focal{i}.bmp")
    centers.append(find_center.find_center_method2(img, r=20, n=1, strike=1))

### 准确率计算

In [ ]:
cam_size = [1200,1920]
detect = cdp.IntensitySumInRegions(size=cam_size, centers=centers, class_region_pixels=mps['class_region_pixels'], device='cpu')
plt.figure()
ax = plt.subplot()
ax.imshow(torch.zeros(cam_size))
detect.show_boundaries(ax)
plt.show()
folder_path = os.path.abspath(region_path)
image_files = glob.glob(os.path.join(folder_path, '*.bmp'))
detectors, pred, label = cdp.predict(image_files, detect)
acc = cdp.acc_cal(label, pred)

# 单张显示

In [ ]:
temp_path1 = ""
slm.Set_Offset(screenNum=1, offset_x=0, offset_y=0)
slm.Disp_ReadImage(path=os.path.abspath(temp_path1), screenNum=1, bits=8)

In [ ]:
temp_path2 = ""
i = Image.open(temp_path2)
imgs.append(np.array(i, dtype=np.uint8))
dlp.stopsequence()  # 如果正在运行，需要先停止。
exposure_time = [100000]*len(imgs)  # 单位us
dark_time=[0]*len(imgs)        # 单位us,如果要用触发模式，darktime不能为0。
trigger_in=[False]*len(imgs)
trigger_out=[1]*len(imgs)   # 好像没用，设置为0或者1都会有硬件触发输出。
rep = 0  # 是显示多少张图，而不是整个序列重复多少次。
dlp.defsequence(images=imgs,exp=exposure_time,ti=trigger_in, dt=dark_time,to=trigger_out,rep=rep)
dlp.startsequence()

# 结束

In [ ]:
# stop acquisition
cam.stream_off()
# close device
cam.close_device()
dlp.stopsequence()
slm.Close_window(1)